## Create a new project

It is always good idea to keep the projects seperate. This function creates a new project with subdirectories and a basic configuration file in the user defined directory otherwise the project is created in the current working directory.

You can always add new videos to the project at any stage of the project. 

In [1]:
import deeplabcut

/home/gosztolai/anaconda3/envs/DLC_env/lib/python3.6/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
import os
print(os.getcwd())
import matplotlib
%matplotlib inline

/data/LiftFly3D/prism/side_view


In [3]:

task='sideJointTracking' # Enter the name of your experiment Task
experimenter='PrismData' # Enter the name of the experimenter
videofile_path = [
    "/data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly1_001_prism.mp4",
    "/data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly1_002_prism.mp4",
    "/data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly1_003_prism.mp4",
    "/data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly1_004_prism.mp4",
    "/data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly2_001_prism.mp4",
    "/data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly2_002_prism.mp4",
    "/data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly2_003_prism.mp4",
    "/data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly2_004_prism.mp4"
]

deeplabcut_dir = "/data/LiftFly3D/prism/side_view/"
config_path = deeplabcut_dir + "config.yaml"

In [ ]:
deeplabcut.create_new_project(task, experimenter, data_dirs)

## Extract frames from videos 
A key point for a successful feature detector is to select diverse frames, which are typical for the behavior you study that should be labeled.

This function selects N frames either uniformly sampled from a particular video (or folder) (algo=='uniform'). Note: this might not yield diverse frames, if the behavior is sparsely distributed (consider using kmeans), and/or select frames manually etc.

Also make sure to get select data from different (behavioral) sessions and different animals if those vary substantially (to train an invariant feature detector).

Individual images should not be too big (i.e. < 850 x 850 pixel). Although this can be taken care of later as well, it is advisable to crop the frames, to remove unnecessary parts of the frame as much as possible.

Always check the output of cropping. If you are happy with the results proceed to labeling.

In [ ]:
'''
deeplabcut.extract_frames(config_path, 'automatic', 'kmeans', crop=False, userfeedback=True)
# there are other ways to grab frames, such as by clustering 'kmeans'; please see the paper.
'''

## Label the extracted frames
Only videos in the config file can be used to extract the frames. Extracted labels for each video are stored in the project directory under the subdirectory **'labeled-data'**. Each subdirectory is named after the name of the video. The toolbox has a labeling toolbox which could be used for labeling. 

In [ ]:
#'''
%gui wx
deeplabcut.label_frames(config_path)
#'''

## Create a training dataset
This function generates the training data information for DeepCut (which requires a mat file) based on the pandas dataframes that hold label information. The user can set the fraction of the training set size (from all labeled image in the hd5 file) in the config.yaml file. While creating the dataset, the user can create multiple shuffles. 

After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**.

Now it is the time to start training the network!

In [ ]:
deeplabcut.create_training_dataset(config_path)

## Start training

This function trains the network for a specific shuffle of the training dataset. 

In [ ]:
import tensorflow as tf
tf.test.is_gpu_available(
    cuda_only=False,
    min_cuda_compute_capability=None
)

In [ ]:
'''
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.5)

sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

deeplabcut.train_network(config_path,
                         displayiters=2000,
                         saveiters=20000,
                         maxiters=200000)
'''

## Start evaluating
This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results**

In [ ]:
deeplabcut.evaluate_network(config_path, plotting=True)

## Start Analyzing videos
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides. 

In [6]:
deeplabcut.analyze_videos(config_path,videofile_path, videotype='.mp4')

Using snapshot-200000 for model /data/LiftFly3D/prism/side_view/dlc-models/iteration-0/sideJointTrackingDec17-trainset95shuffle1
Initializing ResNet
INFO:tensorflow:Restoring parameters from /data/LiftFly3D/prism/side_view/dlc-models/iteration-0/sideJointTrackingDec17-trainset95shuffle1/train/snapshot-200000


INFO:tensorflow:Restoring parameters from /data/LiftFly3D/prism/side_view/dlc-models/iteration-0/sideJointTrackingDec17-trainset95shuffle1/train/snapshot-200000
  0%|          | 0/6022 [00:00<?, ?it/s]

Starting to analyze %  /data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly1_001_prism.mp4
Loading  /data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly1_001_prism.mp4
Duration of video [s]:  240.88 , recorded with  25.0 fps!
Overall # of frames:  6022  found with (before cropping) frame dimensions:  550 260
Starting to extract posture


6060it [00:44, 135.61it/s]                          
  0%|          | 0/12320 [00:00<?, ?it/s]

Detected frames:  6022
Saving results in /data/LiftFly3D/prism/side_view/videos...
Starting to analyze %  /data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly1_002_prism.mp4
Loading  /data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly1_002_prism.mp4
Duration of video [s]:  492.8 , recorded with  25.0 fps!
Overall # of frames:  12320  found with (before cropping) frame dimensions:  550 260
Starting to extract posture


...

3434it [00:26, 127.83it/s]                          
  0%|          | 0/2616 [00:00<?, ?it/s]

Detected frames:  3418
Saving results in /data/LiftFly3D/prism/side_view/videos...
Starting to analyze %  /data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly2_004_prism.mp4
Loading  /data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly2_004_prism.mp4
Duration of video [s]:  104.64 , recorded with  25.0 fps!
Overall # of frames:  2616  found with (before cropping) frame dimensions:  550 260
Starting to extract posture


2626it [00:20, 128.70it/s]                          

Detected frames:  2616
Saving results in /data/LiftFly3D/prism/side_view/videos...
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract any outlier frames!


'DLC_resnet50_sideJointTrackingDec17shuffle1_200000'

## Extract outlier frames [optional step]
This is an optional step and is used only when the evaluation results are poor i.e. the labels are incorrectly predicted. In such a case, the user can use the following function to extract frames where the labels are incorrectly predicted. Make sure to provide the correct value of the "iterations" as it will be used to create the unique directory where the extracted frames will be saved.

In [ ]:
path_config_file = '/home/morales/Documents/Morales/fly_lane-DM-2019-02-19/config.yaml'
crop_video = '/home/morales/Documents/Morales/MDN3_Screen/GFP/190315/161441_s0a0_p75/output_40fps_5.avi'

In [ ]:
path_config_file = '/home/morales/Documents/Morales/fly_lane-DM-2019-02-19/config.yaml'
crop_video = '/home/morales/Documents/Morales/MDN3_Screen/GFP/190315/161441_s0a0_p75/output_40fps_5.avi'

deeplabcut.extract_outlier_frames(path_config_file,[crop_video])

## Refine Labels [optional step]
Following the extraction of outlier frames, the user can use the following function to move the predicted labels to the correct location. Thus augmenting the training dataset.

In [ ]:
%gui wx
deeplabcut.refine_labels(path_config_file)

In [ ]:
#Once all folders are relabeled, check them and advance. See how to check labels, above!
deeplabcut.merge_datasets(path_config_file)

## Create a new iteration of training dataset [optional step]
Following the refine labels, append these frames to the original dataset to create a new iteration of training dataset.

In [ ]:
deeplabcut.create_training_dataset(path_config_file)

## Create labeled video
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

In [7]:
deeplabcut.create_labeled_video(config_path,videofile_path)

  1%|          | 46/6022 [00:00<00:13, 455.29it/s]

Starting %  /data/LiftFly3D/prism/side_view/videos ['/data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly1_001_prism.mp4', '/data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly1_002_prism.mp4', '/data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly1_003_prism.mp4', '/data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly1_004_prism.mp4', '/data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly2_001_prism.mp4', '/data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly2_002_prism.mp4', '/data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly2_003_prism.mp4', '/data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly2_004_prism.mp4']
Loading  /data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly1_001_prism.mp4 and data.
6022
Duration of video [s]:  240.88 , recorded with  25.0 fps!
Overall # of frames:  6022 with cropped frame dimensions:  550 260
Generating frames and creating video.


  0%|          | 41/12320 [00:00<00:30, 398.18it/s]

Starting %  /data/LiftFly3D/prism/side_view/videos ['/data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly1_001_prism.mp4', '/data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly1_002_prism.mp4', '/data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly1_003_prism.mp4', '/data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly1_004_prism.mp4', '/data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly2_001_prism.mp4', '/data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly2_002_prism.mp4', '/data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly2_003_prism.mp4', '/data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly2_004_prism.mp4']
Loading  /data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly1_002_prism.mp4 and data.
12320
Duration of video [s]:  492.8 , recorded with  25.0 fps!
Overall # of frames:  12320 with cropped frame dimensions:  550 260
Generating frames and creating video.


  1%|          | 39/7110 [00:00<00:18, 385.09it/s]

...

  1%|          | 41/3418 [00:00<00:08, 407.37it/s]

Starting %  /data/LiftFly3D/prism/side_view/videos ['/data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly1_001_prism.mp4', '/data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly1_002_prism.mp4', '/data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly1_003_prism.mp4', '/data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly1_004_prism.mp4', '/data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly2_001_prism.mp4', '/data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly2_002_prism.mp4', '/data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly2_003_prism.mp4', '/data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly2_004_prism.mp4']
Loading  /data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly2_003_prism.mp4 and data.
3418
Duration of video [s]:  136.72 , recorded with  25.0 fps!
Overall # of frames:  3418 with cropped frame dimensions:  550 260
Generating frames and creating video.


  1%|▏         | 38/2616 [00:00<00:06, 374.74it/s]

Starting %  /data/LiftFly3D/prism/side_view/videos ['/data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly1_001_prism.mp4', '/data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly1_002_prism.mp4', '/data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly1_003_prism.mp4', '/data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly1_004_prism.mp4', '/data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly2_001_prism.mp4', '/data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly2_002_prism.mp4', '/data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly2_003_prism.mp4', '/data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly2_004_prism.mp4']
Loading  /data/LiftFly3D/prism/side_view/videos/video_191125_PR_Fly2_004_prism.mp4 and data.
2616
Duration of video [s]:  104.64 , recorded with  25.0 fps!
Overall # of frames:  2616 with cropped frame dimensions:  550 260
Generating frames and creating video.


100%|██████████| 2616/2616 [00:05<00:00, 442.45it/s]


## Plot the trajectories of the analyzed videos
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [ ]:
%matplotlib notebook 
deeplabcut.plot_trajectories(path_config_file,videofile_path)

In [ ]:
sess.close()